# Reto 1
Para el modelo de pregunts y respuestas se basa en patrones prdedefinidos con sus respectivas respuestas, y a través de clasificación el modelo decide cual será la respuesta

# Importación de paqueterías


In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [3]:
import pandas as pd
import numpy as np
import re
import json 
import string
import torch
import nltk
import gensim
from nltk import WordNetLemmatizer
from sklearn.metrics import accuracy_score, average_precision_score
import torch.nn as nn
import torchtext
from torchtext.data import get_tokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

# Carga de datos
Conjunto de datos para entrenar el modelo, con respuestas básicas

In [5]:
f = open('/content/Intent.json')
intents = json.load(f)

# Identificación de variable objetivo y características (preprocesamiento)
Los datos tienen diferentes _intenciones_ y respuestas de acuerdo a la intención

In [6]:
def preprocesamiento(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [7]:
# listas para las palabras tokenizadas, las clases que identifican a cada respuesta, las características y la variable objetivo, en ese orden respectivamente
w = [] # para la representación del lenguaje
clases = [] # para guardar las etiquetas
X = [] # guardamos el text con el que se entrena el modelo
y = [] # para cada etiqueta que corresponde a cada X

In [8]:
for intent in intents['intents']:
  for response in intent['responses']:
    tokens = nltk.word_tokenize(response)
    w.extend(tokens)
    X.append(response)
    y.append(intent['text'])

  if intent['text'] not in clases:
    clases.append(intent['text'])

*Lematizamos* como parte del preprocesamiento, lemmatizer convierte su palabra a su forma canonica. por ejemplo, las palabras [change, changes, changing, changed] pasan a ser 'change'

In [11]:
lemmatizer = WordNetLemmatizer()
palabras = [lemmatizer.lemmatize(word.lower()) for word in w if word not in string.punctuation]

palabras = sorted(set(palabras))
# clases = sorted(set(clases))

# Creación de la representación del lenguaje
De a cuerdo a las restricciones del reto, este modelo es desde cero, por lo tanto usamos una bolsa de palabras (BOW), una técnica de representación de texto que crea un vector con las frecuencias de las palabras

In [12]:
training = []
lista = [0] * len(clases)
lemmatizer = WordNetLemmatizer()

for i, doc in enumerate(X):
  bow = []
  txt = lemmatizer.lemmatize(doc.lower())
  # print(txt)
  for word in w:
    bow.append(1) if word not in txt else bow.append(0)

  out = list(lista)
  out[clases.index(y[i])] = 1
  training.append([bow, out])

np.random.shuffle(training)
training = np.array(training)

# X_train = torch.FloatTensor(list(training[:, 0]))
# y_train = torch.FloatTensor(list(training[:, 1]))
X_train = np.array(list(training[:, 0]))
y_train = np.array(list(training[:, 1]))

<ipython-input-12-6c2703a97f8b>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [13]:
(X_train[0]).shape

(8490,)

In [14]:
y_train[0].shape

(22,)

Creamos la un perceptrón multicapas, al ser una representación no tan avanzada del lenguaje, no requerimos una red recurrente, en el codigo explicamos linea por linea

In [15]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu')) # creamos una capa de entrada con activación ReLU
model.add(Dense(64, activation='relu')) # la capa oculta, con activación ReLU
model.add(Dense(len(y_train[0]), activation='softmax')) # y al ultima capa tiene una activación softmax

# para el optimizador usamos ADAM, 
#pero se puede usar cualquier variante del descenso por el gradiente
adam = tf.keras.optimizers.Adam(learning_rate=0.01) 

model.compile(loss='categorical_crossentropy', # usamos categorical_crossentropy porque nuestro problemas es de clasificacion
              optimizer=adam,
              metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1086848   
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 22)                1430      
                                                                 
Total params: 1,096,534
Trainable params: 1,096,534
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.fit(x=X_train, y=y_train, epochs=150, verbose=1)

Epoch 1/150
10/10 [==============================] - 1s 23ms/step - loss: 51.8190 - accuracy: 0.3141
Epoch 2/150
10/10 [==============================] - 0s 20ms/step - loss: 8.2605 - accuracy: 0.5545
Epoch 3/150
10/10 [==============================] - 0s 19ms/step - loss: 2.5817 - accuracy: 0.7372
Epoch 4/150
10/10 [==============================] - 0s 17ms/step - loss: 1.1567 - accuracy: 0.7500
Epoch 5/150
10/10 [==============================] - 0s 17ms/step - loss: 0.7673 - accuracy: 0.7885
Epoch 6/150
10/10 [==============================] - 0s 19ms/step - loss: 0.7294 - accuracy: 0.7821
Epoch 7/150
10/10 [==============================] - 0s 19ms/step - loss: 0.6566 - accuracy: 0.7949
Epoch 8/150
10/10 [==============================] - 0s 17ms/step - loss: 0.6002 - accuracy: 0.7949
Epoch 9/150
10/10 [==============================] - 0s 18ms/step - loss: 0.5865 - accuracy: 0.8205
Epoch 10/150
10/10 [==============================] - 0s 17ms/step - loss: 0.5782 - accuracy: 0.814

In [26]:
# limpiamos el texto que introduzca el usuario
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

# creamos una bolsa de palabras para la entrada
def bag_of_words(text, voc):
  tok = clean_text(text)
  bow = [0] * len(voc)
  for w in tok:
    for i, w in enumerate(voc):
      if word == w:
        bow[i] = 1
  return np.array(bow)

# asignamos clases con la bolsa de palabras
def clase(text, voc, labels):
  bow = bag_of_words(text, voc)
  pred = model.predict(np.array([bow]))[0]
  thresh = 0.5
  y_pred = [[i, res] for i, res in enumerate(pred) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)

  lista = []
  for r in y_pred:
    lista.append(labels[r[0]])
  return lista

# creamos la pregunta y respuesta 
def pregunta_respuesta(intenciones, intenciones_json):
  if len(intenciones) == 0:
    result = "Pregunta de nuevo"
  else:
    tag = intenciones[0]
    lista_intenciones = intenciones_json['intents']

    for intent in lista_intenciones:
      if i[intent] ==tag:
        result = np.random.choice(intent['text'])
        break

  return result 

In [25]:
while True:
  pregunta = input('')
  if pregunta == "0":
    break

  intents = clase(pregunta, palabras, clase)
  res = pregunta_respuesta(intents, )
  print(res)

hola


ValueError: ignored